### Predicting the CDR - The Clinical Dementia Rating. Higher the number, higher the chances of dementia

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
df = pd.read_csv("OASIS4_data_CDR.csv")

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt

# Your plotting code here

plt.show()  # Display the plot in the Jupyter Notebook output


In [4]:
# # Import the necessary module
# from ydata_profiling import ProfileReport

# # Create a ProfileReport object
# report = ProfileReport(df)

# # Generate the data profiling report
# report.to_file('oasis_4_report.html')

In [5]:
#print number of rows and columns
print(df.shape)

(1158, 11)


In [6]:
# impute the missing values of columns 3 onwards with the mean of the respective columns, excpet column 'cdr'
df.iloc[:,2:-1] = df.iloc[:,2:-1].fillna(df.iloc[:,2:-1].mean())

In [7]:
df.isnull().sum()

oasis_id       0
cdr_id         0
visit_days     0
memory         0
orient         0
judgement      0
community      0
homehobb       0
perscare       0
sumbox         0
cdr           10
dtype: int64

In [8]:
# remove the null values from 'cdr'
df = df.dropna()

In [9]:
# check all the columns in df and convert all the int columns to float, except 'cdr'
for col in df.columns:
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1148 entries, 0 to 1157
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   oasis_id    1148 non-null   object 
 1   cdr_id      1148 non-null   object 
 2   visit_days  1148 non-null   float64
 3   memory      1148 non-null   float64
 4   orient      1148 non-null   float64
 5   judgement   1148 non-null   float64
 6   community   1148 non-null   float64
 7   homehobb    1148 non-null   float64
 8   perscare    1148 non-null   float64
 9   sumbox      1148 non-null   float64
 10  cdr         1148 non-null   float64
dtypes: float64(9), object(2)
memory usage: 107.6+ KB


In [11]:
df = df.drop(['oasis_id', 'cdr_id'], axis=1)

#### Encoding

In [12]:
# convert contents of 'cdr' column to string
df['cdr'] = df['cdr'].astype('str')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1148 entries, 0 to 1157
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   visit_days  1148 non-null   float64
 1   memory      1148 non-null   float64
 2   orient      1148 non-null   float64
 3   judgement   1148 non-null   float64
 4   community   1148 non-null   float64
 5   homehobb    1148 non-null   float64
 6   perscare    1148 non-null   float64
 7   sumbox      1148 non-null   float64
 8   cdr         1148 non-null   object 
dtypes: float64(8), object(1)
memory usage: 89.7+ KB


In [14]:
# print count of each type of values 'cdr'
print(df['cdr'].value_counts())

0.5    654
1.0    294
0.0    102
2.0     96
3.0      2
Name: cdr, dtype: int64


In [15]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['cdr'] = le.fit_transform(df['cdr'])

In [16]:
print(df['cdr'])

0       0.5
1       0.5
2       0.5
3       0.5
4       0.5
       ... 
1153    0.5
1154    0.5
1155    0.5
1156    0.5
1157    0.5
Name: cdr, Length: 1148, dtype: object


#### Splitting into Train and Test

In [27]:
# split into x and y where y is 'Group' 
x = df.drop(columns=['cdr'])
y = df['cdr']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2, random_state=42, stratify=y)

#### Standad Scaler

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_subset = x_train.iloc[:, 2:]
x_train_scaled = scaler.fit_transform(x_train_subset)
x_train.iloc[:, 2:] = x_train_scaled

x_test_subset = x_test.iloc[:, 2:]
x_test_scaled = scaler.transform(x_test_subset)
x_test.iloc[:, 2:] = x_test_scaled


#### Model Training

##### Linear Regression

In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)
predictions = model.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")
conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.94
Confusion Matrix:
[[ 18   3   0   0]
 [  0 130   0   1]
 [  0   7  51   1]
 [  0   2   0  17]]


C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### Logistic Regression after smote

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

# Perform SMOTE with reduced k_neighbors
smote = SMOTE(sampling_strategy='auto', k_neighbors=1)  # Adjust k_neighbors as needed
X_resampled, y_resampled = smote.fit_resample(x, y)

# Split the dataset into training and testing sets
x_train_res, x_test, y_train_res, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(x_train_res, y_train_res)

# Make predictions and evaluate
predictions = model.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")
conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.92
Confusion Matrix:
[[112   8   3   0   0]
 [  7 130  11   0   0]
 [  0   4 120   9   0]
 [  0   7   2 119   0]
 [  0   0   0   0 122]]


C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### SVM

##### RF

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create and train the Random Forest model
model = RandomForestClassifier()
model.fit(x_train, y_train)

# Make predictions
y_pred = model.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.42


##### Gradient Boosting

In [29]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create an XGBoost regressor
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)

# Fit the model on the training data
model.fit(x_train, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test)

# Calculate the mean squared error to evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")


Mean Squared Error: 0.03


In [31]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.94
